# 2. Install required libraries

## Referenece: https://dassum.medium.com/fine-tune-large-language-model-llm-on-a-custom-dataset-with-qlora-fb60abdeba07


In [3]:
!pip install -q -U bitsandbytes transformers peft accelerate datasets scipy einops evaluate trl rouge_score


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 105.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig
)
from tqdm import tqdm
from trl import SFTTrainer
import torch
import time
import pandas as pd
import numpy as np
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Enter your token (input will not be visible): ··········
Add token as git credential? (Y/n) n


In [3]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )

In [4]:
model_name='microsoft/phi-2'
device_map = {"": 0}
original_model = AutoModelForCausalLM.from_pretrained(model_name,
                                                      device_map=device_map,
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [27]:
from datasets import Dataset
import pandas as pd

# Load your CSV file
csv_file_path = "/content/drive/MyDrive/Interview_Bot_fine_tuning/dataset.csv"
df = pd.read_csv(csv_file_path)

df




,Skill,Question,Answer
0,Power BI,How do you create and use Power Query to trans...,NaN
1,Power BI,Can you explain the purpose of DAX in Power BI...,NaN
2,Power BI,What methods do you use to visualize data effe...,NaN
3,Power BI,How do you handle different data sources in Po...,NaN
4,Power BI,What is the significance of hierarchies in Pow...,NaN
...,...,...,...
15610,Microsoft Windows Skills,Network Configuration,Understanding network settings and configurati...
15611,Microsoft Windows Skills,1. Utilize skills tests before interviews to s...,Using skills tests before interviews can help ...
15612,Microsoft Windows Skills,2. Compile and streamline your interview quest...,You only have a limited amount of time to ask ...
15613,Microsoft Windows Skills,3. Ask follow-up questions for deeper insights,Simply using prepared interview questions may ...


In [9]:
df.dropna(inplace=True)
df

,Skill,Question,Answer
36,Power BI,1. How do you handle data modeling in Power BI?,Data modeling in Power BI involves creating re...
37,Power BI,2. Can you explain the difference between Powe...,Power BI Desktop is a Windows-based applicatio...
38,Power BI,3. How do you ensure data security and privacy...,Data security and privacy in Power BI can be e...
39,Power BI,4. What are some common challenges you have fa...,Common challenges in Power BI can include issu...
40,Power BI,5. How do you handle performance optimization ...,Performance optimization in Power BI reports c...
...,...,...,...
15610,Microsoft Windows Skills,Network Configuration,Understanding network settings and configurati...
15611,Microsoft Windows Skills,1. Utilize skills tests before interviews to s...,Using skills tests before interviews can help ...
15612,Microsoft Windows Skills,2. Compile and streamline your interview quest...,You only have a limited amount of time to ask ...
15613,Microsoft Windows Skills,3. Ask follow-up questions for deeper insights,Simply using prepared interview questions may ...


In [16]:
# Removing prefix numbers from the Question column
df["Question"] = df["Question"].str.replace(r"^\d+\.\s*", "", regex=True)

In [17]:
df

,Skill,Question,Answer
36,Power BI,How do you handle data modeling in Power BI?,Data modeling in Power BI involves creating re...
37,Power BI,Can you explain the difference between Power B...,Power BI Desktop is a Windows-based applicatio...
38,Power BI,How do you ensure data security and privacy in...,Data security and privacy in Power BI can be e...
39,Power BI,What are some common challenges you have faced...,Common challenges in Power BI can include issu...
40,Power BI,How do you handle performance optimization in ...,Performance optimization in Power BI reports c...
...,...,...,...
15610,Microsoft Windows Skills,Network Configuration,Understanding network settings and configurati...
15611,Microsoft Windows Skills,Utilize skills tests before interviews to scre...,Using skills tests before interviews can help ...
15612,Microsoft Windows Skills,Compile and streamline your interview questions,You only have a limited amount of time to ask ...
15613,Microsoft Windows Skills,Ask follow-up questions for deeper insights,Simply using prepared interview questions may ...


In [64]:
df.to_csv("/content/drive/MyDrive/Interview_Bot_fine_tuning/cleaned_dataset.csv", index=False)

In [63]:
# Combining Skill and Question columns
df["Question"] = df["Skill"] + ": " + df["Question"]
df

,Skill,Question,Answer
0,Power BI,Power BI: How do you handle data modeling in P...,Data modeling in Power BI involves creating re...
1,Power BI,Power BI: Can you explain the difference betwe...,Power BI Desktop is a Windows-based applicatio...
2,Power BI,Power BI: How do you ensure data security and ...,Data security and privacy in Power BI can be e...
3,Power BI,Power BI: What are some common challenges you ...,Common challenges in Power BI can include issu...
4,Power BI,Power BI: How do you handle performance optimi...,Performance optimization in Power BI reports c...
...,...,...,...
5549,Microsoft Windows Skills,Microsoft Windows Skills: Network Configuration,Understanding network settings and configurati...
5550,Microsoft Windows Skills,Microsoft Windows Skills: Utilize skills tests...,Using skills tests before interviews can help ...
5551,Microsoft Windows Skills,Microsoft Windows Skills: Compile and streamli...,You only have a limited amount of time to ask ...
5552,Microsoft Windows Skills,Microsoft Windows Skills: Ask follow-up questi...,Simply using prepared interview questions may ...


### Cleaned Data is ready Start loading the data from Here

In [5]:
df= pd.read_csv("/content/drive/MyDrive/Interview_Bot_fine_tuning/cleaned_dataset.csv")
df

,Skill,Question,Answer
0,Power BI,Power BI: How do you handle data modeling in P...,Data modeling in Power BI involves creating re...
1,Power BI,Power BI: Can you explain the difference betwe...,Power BI Desktop is a Windows-based applicatio...
2,Power BI,Power BI: How do you ensure data security and ...,Data security and privacy in Power BI can be e...
3,Power BI,Power BI: What are some common challenges you ...,Common challenges in Power BI can include issu...
4,Power BI,Power BI: How do you handle performance optimi...,Performance optimization in Power BI reports c...
...,...,...,...
5549,Microsoft Windows Skills,Microsoft Windows Skills: Network Configuration,Understanding network settings and configurati...
5550,Microsoft Windows Skills,Microsoft Windows Skills: Utilize skills tests...,Using skills tests before interviews can help ...
5551,Microsoft Windows Skills,Microsoft Windows Skills: Compile and streamli...,You only have a limited amount of time to ask ...
5552,Microsoft Windows Skills,Microsoft Windows Skills: Ask follow-up questi...,Simply using prepared interview questions may ...


In [6]:
# Splitting the dataset into train, validation, and test

from datasets import Dataset, DatasetDict


train_df = df.sample(frac=0.7, random_state=42)
remaining_df = df.drop(train_df.index)
validation_df = remaining_df.sample(frac=0.5, random_state=42)
test_df = remaining_df.drop(validation_df.index)

# Converting DataFrames to Hugging Face Dataset format
dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(validation_df),
    "test": Dataset.from_pandas(test_df)
})

# Display dataset structure
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['Skill', 'Question', 'Answer', '__index_level_0__'],
        num_rows: 3888
    })
    validation: Dataset({
        features: ['Skill', 'Question', 'Answer', '__index_level_0__'],
        num_rows: 833
    })
    test: Dataset({
        features: ['Skill', 'Question', 'Answer', '__index_level_0__'],
        num_rows: 833
    })
})


In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Skill', 'Question', 'Answer', '__index_level_0__'],
        num_rows: 3888
    })
    validation: Dataset({
        features: ['Skill', 'Question', 'Answer', '__index_level_0__'],
        num_rows: 833
    })
    test: Dataset({
        features: ['Skill', 'Question', 'Answer', '__index_level_0__'],
        num_rows: 833
    })
})

In [8]:
dataset['train'][0]

{'Skill': 'OpenCV',
 'Question': 'OpenCV: Feature Detection and Matching',
 'Answer': 'Feature detection and matching are essential for tasks like object recognition and tracking. These skills are valuable for developers working on advanced computer vision projects.',
 '__index_level_0__': 4817}

In [9]:
dataset['train'][0]['Question']

'OpenCV: Feature Detection and Matching'

In [10]:
dataset['train'][0]['Answer']

'Feature detection and matching are essential for tasks like object recognition and tracking. These skills are valuable for developers working on advanced computer vision projects.'

In [11]:
## Tokenization

tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True,padding_side="left",add_eos_token=True,add_bos_token=True,use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

In [12]:
## Test the Model with Zero Shot Inferencing

%%time
from transformers import set_seed, pipeline

seed = 42
set_seed(seed)

index = 10

question = dataset['test'][index]['Question']
answer = dataset['test'][index]['Answer']

formatted_prompt = f"Instruct: Answer the following Question.\n{question}\nOutput:\n"

# Create a text generation pipeline using your model
generator = pipeline("text-generation", model=original_model, tokenizer=tokenizer)

# Generate text using the pipeline
res = generator(formatted_prompt, max_new_tokens=100)

#print(res[0])
output = res[0]['generated_text'].split('Output:\n')[1]

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{formatted_prompt}')
print(dash_line)
print(f'BASELINE ANSWER:\n{answer}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

Device set to use cuda:0


---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruct: Answer the following Question.
.NET Core: How does .NET Core handle memory management?
Output:

---------------------------------------------------------------------------------------------------
BASELINE ANSWER:
.NET Core handles memory management through a process called garbage collection. This automated system manages the allocation and release of memory for your application, preventing memory leaks and ensuring that unused objects are removed from memory. Garbage collection in .NET Core is efficient and works in the background, allowing developers to focus on writing code without worrying excessively about memory management.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
.NET Core handles memory management by using a garbage collector to automatically reclaim memory that is no

## As you can see there are similarities between the answers but we will got for finetuning the model

In [13]:
def create_prompt_formats(sample):
    """
    Format various fields of the sample ('instruction','output')
    Then concatenate them using two newline characters
    :param sample: Sample dictionnary
    """
    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruct: Answer the below question."
    RESPONSE_KEY = "### Output:"
    END_KEY = "### End"

    blurb = f"\n{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}"
    input_context = f"{sample['Question']}" if sample["Question"] else None
    response = f"{RESPONSE_KEY}\n{sample['Answer']}"
    end = f"{END_KEY}"

    parts = [part for part in [blurb, instruction, input_context, response, end] if part]

    formatted_prompt = "\n\n".join(parts)
    sample["text"] = formatted_prompt

    return sample

In [14]:
from functools import partial

# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length


def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizing a batch
    """
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )


In [15]:
# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int,seed, dataset):
    """Format & tokenize it so it is ready for training
    :param tokenizer (AutoTokenizer): Model Tokenizer
    :param max_length (int): Maximum number of tokens to emit from tokenizer
    """

    # Add prompt to each sample
    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats)#, batched=True)

    # Apply preprocessing to each batch of the dataset & and remove 'instruction', 'context', 'response', 'category' fields
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=['Skill', '__index_level_0__', 'Question', 'Answer'],
    )

    # Filter out samples that have input_ids exceeding max_length
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)

    # Shuffle dataset
    dataset = dataset.shuffle(seed=seed)

    return dataset

In [16]:
## Pre-process dataset
max_length = get_max_length(original_model)
print(max_length)

train_dataset = preprocess_dataset(tokenizer, max_length,seed, dataset['train'])
eval_dataset = preprocess_dataset(tokenizer, max_length,seed, dataset['validation'])

Found max lenth: 2048
2048
Preprocessing dataset...


Map:   0%|          | 0/3888 [00:00<?, ? examples/s]

Map:   0%|          | 0/3888 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3888 [00:00<?, ? examples/s]

Preprocessing dataset...


Map:   0%|          | 0/833 [00:00<?, ? examples/s]

Map:   0%|          | 0/833 [00:00<?, ? examples/s]

Filter:   0%|          | 0/833 [00:00<?, ? examples/s]

In [17]:
train_dataset

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 3888
})

In [18]:
train_dataset['text']

['\nBelow is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruct: Answer the below question.\n\nAzure: Describe the process of implementing a blue-green deployment in Azure.\n\n### Output:\nA candidate with deployment experience should outline the following steps:\n\n### End',
 "\nBelow is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruct: Answer the below question.\n\nFlask: How do you ensure your Flask tests are maintainable?\n\n### Output:\nEnsuring maintainable tests involves writing clear and concise test cases that are easy to understand and modify. Tests should be organized in a logical structure, and any setup or teardown code should be reused through fixtures or helper functions. It's also important to keep tests up to date with changes in the application. Regularly reviewing and refactoring tests helps maintain their relevance and effectiveness.\n\n### E

## Preparing the model for QLoRA

In [94]:
# 2 - Using the prepare_model_for_kbit_training method from PEFT
# Preparing the Model for QLoRA


##original_model = prepare_model_for_kbit_training(original_model)

## Setup PEFT for Fine-Tuning

In [19]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

config = LoraConfig(
    r=32, #Rank
    lora_alpha=32,
    target_modules=[
        'q_proj',
        'k_proj',
        'v_proj',
        'dense'
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

# 1 - Enabling gradient checkpointing to reduce memory usage during fine-tuning
original_model.gradient_checkpointing_enable()

#Before applying LoRA, ensure the model is prepped for quantization-aware training:
peft_model = prepare_model_for_kbit_training(original_model)  # Add this step
peft_model = get_peft_model(original_model, config)

for param in peft_model.parameters():
    if param.dtype in [torch.float16, torch.float32, torch.bfloat16]:
        param.requires_grad = True


In [22]:
# Function to count total and trainable parameters
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    percent_trainable = (trainable_params / total_params) * 100
    return total_params, trainable_params, percent_trainable

# Before applying LoRA
total_params_before, trainable_params_before, percent_trainable_before = count_parameters(original_model)
print(f"Before LoRA:")
print(f"Total Parameters: {total_params_before:,}")
print(f"Trainable Parameters: {trainable_params_before:,}")
print(f"Percentage of Trainable Parameters: {percent_trainable_before:.4f}%\n")

# After applying LoRA
total_params_after, trainable_params_after, percent_trainable_after = count_parameters(peft_model)
print(f"After LoRA:")
print(f"Total Parameters: {total_params_after:,}")
print(f"Trainable Parameters: {trainable_params_after:,}")
print(f"Percentage of Trainable Parameters: {percent_trainable_after:.4f}%")



Before LoRA:
Total Parameters: 1,542,364,160
Trainable Parameters: 284,072,960
Percentage of Trainable Parameters: 18.4180%

After LoRA:
Total Parameters: 1,542,364,160
Trainable Parameters: 284,072,960
Percentage of Trainable Parameters: 18.4180%


In [21]:
output_dir = f'./peft-interview-bot-training-{str(int(time.time()))}'
output_dir

'./peft-interview-bot-training-1742234367'

In [23]:
import transformers

peft_training_args = TrainingArguments(
    output_dir = output_dir,
    warmup_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    max_steps=450,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    logging_steps=25,
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=25,
    evaluation_strategy="steps",
    eval_steps=25,
    do_eval=True,
    gradient_checkpointing=True,
    report_to="none",
    overwrite_output_dir = 'True',
    group_by_length=True,
)

peft_model.config.use_cache = False

peft_trainer = transformers.Trainer(
    model=peft_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=peft_training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [24]:
peft_trainer.train()


Step,Training Loss,Validation Loss
25,1.503500,1.257159
50,1.192100,1.182314
75,1.187300,1.109061
100,1.009300,1.110946
125,1.252100,1.051729
150,0.958500,1.065248
175,1.247000,1.029889
200,0.868400,1.038612
225,1.205300,1.008965
250,0.815000,1.008038


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

TrainOutput(global_step=450, training_loss=1.0699381679958768, metrics={'train_runtime': 2207.1381, 'train_samples_per_second': 0.816, 'train_steps_per_second': 0.204, 'total_flos': 3605283006013440.0, 'train_loss': 1.0699381679958768, 'epoch': 0.46296296296296297})

In [55]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

base_model_id = "microsoft/phi-2"
base_model = AutoModelForCausalLM.from_pretrained(base_model_id,
                                                      device_map='auto',
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [56]:
eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token


In [57]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "/content/peft-interview-bot-training-1742234367/checkpoint-450",torch_dtype=torch.float16,is_trainable=False)

In [58]:
merged_model = ft_model.merge_and_unload()


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:355: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [60]:
# ✅ Inference Function
def generate_response(prompt, model, tokenizer):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_length=200)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [62]:
# ✅ Run Inference
input_text = "Can you explain the difference between Power BI Desktop and Power BI Service?"
response = generate_response(input_text, merged_model, eval_tokenizer)

print("Model Response:", response)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Model Response: Can you explain the difference between Power BI Desktop and Power BI Service?
Answer: Power BI Desktop is a desktop application that allows users to create, visualize, and analyze data. It provides a user-friendly interface and allows users to create interactive dashboards and reports. Power BI Service, on the other hand, is a cloud-based service that allows users to create and deploy data visualizations and reports. It provides a more scalable and flexible solution for organizations that need to analyze large amounts of data.



In [69]:
# ✅ Save the merged model
save_directory = "/content/drive/MyDrive/Interview_Bot_fine_tuning/fine_tuned_phi2"
merged_model.save_pretrained(save_directory)

eval_tokenizer.save_pretrained(save_directory)

print(f"Merged model saved to {save_directory}")



Merged model saved to /content/drive/MyDrive/Interview_Bot_fine_tuning/fine_tuned_phi2


In [70]:
# ✅ Load the saved model for inference
loaded_model = AutoModelForCausalLM.from_pretrained(save_directory)
loaded_tokenizer = AutoTokenizer.from_pretrained(save_directory, use_fast=False)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [71]:
# ✅ Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
loaded_model.to(device)


PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (dense): Linear4bit(in_features=2560, out_features=2560, bias=True)
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear4bit(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear4bit(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (rotary_emb): PhiRotaryEmbedding()
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (final_layernorm): 

In [72]:
# ✅ Inference Function
def generate_response(prompt, model, tokenizer):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_length=200)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [74]:
# ✅ Run Inference
input_text = "Can you explain the difference between Power BI Desktop and Power BI Service?"
response = generate_response(input_text, loaded_model, loaded_tokenizer)

print("Model Response:", response)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Model Response: Can you explain the difference between Power BI Desktop and Power BI Service?
Answer: Power BI Desktop is a desktop application that allows users to create, visualize, and analyze data. It provides a user-friendly interface and allows users to create interactive dashboards and reports. Power BI Service, on the other hand, is a cloud-based service that allows users to create and deploy data visualizations and reports. It provides a more scalable and flexible solution for organizations that need to analyze large amounts of data.



## Push Model to huggingface hub

In [1]:
from huggingface_hub import login

login()


In [10]:
!pip install -U bitsandbytes

In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import HfApi

# Define your Hugging Face model repository
hf_repo_id = "sachin19566/interview-bot-finetuned-phi-2"  # Change this to your repo name

# Load the merged model & tokenizer
save_directory = "/content/drive/MyDrive/Interview_Bot_fine_tuning/fine_tuned_phi2"

model = AutoModelForCausalLM.from_pretrained(save_directory)
tokenizer = AutoTokenizer.from_pretrained(save_directory, use_fast=False)

# ✅ Push model & tokenizer to Hugging Face Hub
model.push_to_hub(hf_repo_id)
tokenizer.push_to_hub(hf_repo_id)

print(f"Model successfully uploaded to: https://huggingface.co/{hf_repo_id}")


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


ImportError: Using `bitsandbytes` 4-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`